In [12]:
import pandas as pd
data = pd.read_csv("complete_top_10_imputation_rvsd.csv")

In [3]:
data

,Unnamed: 0,new_deceased,population,population_male,population_female,latitude,longitude,area_sq_km,life_expectancy,mobility_workplaces,...,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt_relief,public_information_campaigns,contact_tracing,testing_policy,facial_coverings,vaccination_policy
0,0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,-10.333333,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
1,1,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,-10.333333,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
2,2,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,-10.333333,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
3,3,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,-10.333333,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
4,4,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,-10.333333,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97089,97179,0.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,81.0,-73.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97090,97180,0.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,81.0,-73.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97091,97181,0.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,81.0,-73.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97092,97182,0.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,81.0,-73.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [14]:
num_data = data.select_dtypes(include='number')
# train test split
X = num_data.drop(columns = ['new_confirmed'])
y = num_data['new_confirmed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 45)

In [6]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 17.8 MB/s eta 0:00:00


In [7]:
from catboost import CatBoostRegressor

In [21]:
# print RMSE for default CatBoost model
model_cat = CatBoostRegressor(verbose=False).fit(X_train, y_train)
pred = model_cat.predict(X_test)
print("CstBoost model RMSE = ", np.sqrt(mean_squared_error(pred,y_test)))

CstBoost model RMSE =  2739.8537161679174


In [ ]:
# try to tune CatBoost model for lower RMSE
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
param_grid = {'max_depth': [2,4,6,8,10],
              'num_leaves': [10,20, 31, 40],
              'learning_rate': [0.0001,0.01, 0.05, 0.1,1],
               'reg_lambda':[0, 0.5, 5, 10, 100],
                'n_estimators':[100, 500, 1000,1500],
                'subsample': [0.25,0.5, 0.75, 1.0]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
optimal_params = RandomizedSearchCV(estimator=CatBoostRegressor(random_state=1, verbose=False),
                             param_distributions = param_grid, n_iter = 200,
                             verbose = False,random_state = 1,
                             n_jobs=-1,
                             cv = cv)
optimal_params.fit(X_train, y_train)
print("Optimal parameter values =", optimal_params.best_params_)

In [ ]:
model_cat = CatBoostRegressor(subsample=1, reg_lambda=100, num_leaves= 31, n_estimators=1000, max_depth= 8, learning_rate= 0.1,verbose=False).fit(X_train, y_train)
pred = model_cat.predict(X_test)
print("CstBoost model RMSE = ", np.sqrt(mean_squared_error(pred,y_test)))